In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.datasets import mnist
import matplotlib.pyplot as plt
from network import preprocess_data
import numpy as np
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

In [2]:
def preprocess_data(x: np.ndarray, y: np.ndarray, limit: int): 
    # Shuffle the data
    x_train, _, y_train, _ = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    
    x_train = tf.image.convert_image_dtype(tf.expand_dims(x_train[:limit], axis=-1), dtype='float32') 
    y_train = y_train[:limit]
    
    
    y_train = tf.one_hot(y_train, depth=len(np.unique(y_train)), on_value=1, off_value=0)
    print(x_train.shape)
    print(y_train.shape)
    
    return x_train, y_train

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

np.random.seed(1)
# Limit the data to 2000 and one-hot encoded y
x_train, y_train = preprocess_data(x_train, y_train, 1000)
x_test, y_test = preprocess_data(x_test, y_test, 5000)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

(1000, 28, 28, 1)
(1000, 10)
(5000, 28, 28, 1)
(5000, 10)


2023-12-27 12:59:27.780405: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-12-27 12:59:27.780428: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-27 12:59:27.780437: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-27 12:59:27.780465: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-27 12:59:27.780481: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
def cnn_net(input_shape=(28, 28, 1), n_class=10): 
    input_img = Input(shape=input_shape)
    # CNN's block 1
    Z1 = Conv2D(filters=10, kernel_size=3, strides=1, kernel_initializer='he_normal')(input_img)
    P1 = MaxPooling2D(pool_size=2, strides=2)(Z1)
    A1 = Activation(activation='relu')(P1)
    # CNN's block 2
    Z2 = Conv2D(filters=5, kernel_size=2, strides=1, kernel_initializer='he_normal')(A1)
    P2 = MaxPooling2D(pool_size=2, strides=2)(Z2)
    A2 = Activation(activation='relu')(P2)
    # Dense layer
    F = Flatten()(A2)
    A3 = Dense(units=n_class, kernel_initializer='he_normal', activation='softmax')(F)
    
    model = Model(inputs=input_img, outputs=A3)
    return model

In [5]:
net = cnn_net()
net.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 10)        0         
 D)                                                              
                                                                 
 activation (Activation)     (None, 13, 13, 10)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 5)         205       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 5)           0         
 g2D)                                                        

In [6]:
learning_rate = 0.015  # You can adjust these values
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

net.compile(optimizer='adam', loss=loss_fn,
            metrics=['accuracy'])

In [7]:
history = net.fit(train_dataset, epochs=50)

Epoch 1/50


2023-12-27 12:59:28.508596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 2s 16ms/step - loss: 0.5818 - accuracy: 0.1290
Epoch 2/50
16/16 [==============================] - 0s 9ms/step - loss: 0.3818 - accuracy: 0.1230
Epoch 3/50
16/16 [==============================] - 0s 9ms/step - loss: 0.3302 - accuracy: 0.1680
Epoch 4/50
16/16 [==============================] - 0s 9ms/step - loss: 0.3120 - accuracy: 0.2650
Epoch 5/50
16/16 [==============================] - 0s 9ms/step - loss: 0.2979 - accuracy: 0.3310
Epoch 6/50
16/16 [==============================] - 0s 8ms/step - loss: 0.2832 - accuracy: 0.4120
Epoch 7/50
16/16 [==============================] - 0s 9ms/step - loss: 0.2672 - accuracy: 0.4910
Epoch 8/50
16/16 [==============================] - 0s 8ms/step - loss: 0.2497 - accuracy: 0.5760
Epoch 9/50
16/16 [==============================] - 0s 8ms/step - loss: 0.2303 - accuracy: 0.6390
Epoch 10/50
16/16 [==============================] - 0s 8ms/step - loss: 0.2103 - accuracy: 0.6810
Epoch 11/50
16/16 [==========

In [8]:
loss, accuracy = net.evaluate(test_dataset)
print('Loss', loss)
print('Accuracy', accuracy*100, '%')

79/79 [==============================] - 1s 7ms/step - loss: 0.0644 - accuracy: 0.9208
Loss 0.06441689282655716
Accuracy 92.0799970626831 %
